In [1]:
import os

In [3]:
%pwd

'd:\\sharan_personal\\NLP_GenerativeAI_Projects\\NLP_End_to_End_Text_Summarization\\Text-Summarizer-Project\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'd:\\sharan_personal\\NLP_GenerativeAI_Projects\\NLP_End_to_End_Text_Summarization\\Text-Summarizer-Project'